In [ ]:
import awswrangler as wr
import pandas as pd
import csv
import boto3
import io

s3 = boto3.client("s3")

In [ ]:
dfOriginal = wr.s3.read_parquet(path='s3://cen-ingesta.calidad-raw-dev/replica/reading_detail/', columns=['ID_READING_DETAIL','NOINS','NOINTERVALS','IS_ACTIVE'])


select *  from PRIME_V10.dbo.READING_DETAIL rd
where NOINTERVALS  != 1  and IS_ACTIVE = 1;

In [ ]:
def partir_lista(lista):
    tamano_maximo = 100
    num_sublistas = (len(lista) + tamano_maximo - 1) // tamano_maximo  # Redondeo hacia arriba
    
    sublistas = []
    for i in range(num_sublistas):
        inicio = i * tamano_maximo
        fin = (i + 1) * tamano_maximo
        sublista = lista[inicio:fin]
        sublistas.append(sublista)
    
    return sublistas

In [ ]:
df = dfOriginal.copy()

df = df[(df.NOINTERVALS != 1) & (df.IS_ACTIVE == 1)]
df.drop_duplicates(['NOINS'], inplace=True)

lista = df.NOINS.to_list()

lista = partir_lista(lista)


In [ ]:
csv_file = io.StringIO()
    
# write data to CSV
writer = csv.writer(csv_file)
for row in lista:
    writer.writerow(row)

s3.put_object(Body=csv_file.getvalue(), Bucket='cen-ingesta.calidad-raw-dev', Key='replica/medidores/medidores.csv')

In [ ]:
file = open('result.csv', 'w')

for index,line in enumerate(lista):
    line = str(line)
    line = line.replace("[","(").replace("]",")")
    #line = line.replace("]",")")
    group_name = "Group "+str(index)
    file.write(line+";"+group_name+"\n")
    # Closing file
file.close()

with open('result.csv', 'rb') as file:
    s3.put_object(Body=file, Bucket='cen-ingesta.calidad-raw-dev', Key='replica/medidores/medidores.csv')

